In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4442048 sha256=a93e90130eaba2f61dc378d7ca1412fb0ea0dc533a6d25fa67457f77b7f48e12
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [3]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [4]:
%%writefile tenfile.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = np.random.randint(0,100,size= 1000000)
else :
    data = None

data = comm.scatter(np.array_split(data,size) if rank == 0 else None, root=0)
local_sum = np.sum(data)

total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

if rank == 0:
    print(f"Total sum: {total_sum}")


Writing tenfile.py


In [5]:
!mpirun --allow-run-as-root --oversubscribe -np 8 python tenfile.py

Total sum: 49556742


In [6]:
%%writefile ten.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = np.random.randint(0,1000000,size= 1000000)
else :
    data = None

data = comm.scatter(np.array_split(data,size) if rank == 0 else None, root=0)

local_max = np.max(data)

global_max = comm.reduce(local_max, op=MPI.MAX, root=0)

if rank == 0:
    print(f"Global max: {global_max}")

Writing ten.py


In [7]:
!mpirun --allow-run-as-root --oversubscribe -np 8 python ten.py

Global max: 999999


In [15]:
%%writefile ten1.py
from mpi4py import MPI
import numpy as np

def f(x):
  return np.exp(-x**2)
def integrate_range(start, end, num_points) :
  dx = (end - start) / num_points
  total = 0.0
  for i in range  (num_points) :
    x = start + i * dx
    total += f(x) * dx
  return total

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

num_points = 1000000
points_per_process = num_points // size

local_start = rank * (1.0/size)
local_end = (rank + 1) * (1.0/size)
local_sum = integrate_range(local_start, local_end, points_per_process)

total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)
if rank == 0:
    print(f"Total sum: {total_sum}")

Overwriting ten1.py


In [16]:
!mpirun --allow-run-as-root --oversubscribe -np 10 python ten1.py

Total sum: 0.7468244488726495


In [11]:
%%writefile tinhsopimasterandslaveprocess.py
from mpi4py import MPI
import numpy as np

def f(x):
    return 4.0 / (1.0 + x*x)

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

n = 1000000  # Tổng số điểm chia để tính tích phân
h = 1.0 / n  # Độ rộng mỗi phân đoạn

local_sum = 0.0

# Mỗi tiến trình xử lý các chỉ số i bắt đầu từ rank, bước nhảy là size
for i in range(rank, n, size):
    x = h * (i + 0.5)
    local_sum += f(x)

local_pi = h * local_sum

# Master nhận kết quả từ các slave và cộng lại
pi = comm.reduce(local_pi, op=MPI.SUM, root=0)

if rank == 0:
    print("Giá trị xấp xỉ của pi là:", pi)


Writing tinhsopimasterandslaveprocess.py


In [12]:
!mpirun --allow-run-as-root --oversubscribe -np 8 python tinhsopimasterandslaveprocess.py

Giá trị xấp xỉ của pi là: 3.14159265358989
